In [1]:
from __future__ import division
from __future__ import print_function

import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from utils import load_data, accuracy
from models import GAT, SpGAT
import math
from torch_geometric.datasets import *
import scipy.sparse as sp

In [2]:
adj, features, labels, idx_train, idx_val, idx_test,edges = load_data(seed=0)

Loading cora dataset...


In [3]:
pwd

'/home/zhihao/Document/gnn_fd/graphSage/split/multihead/NC-GNN/gat'

In [4]:
    dataset = Planetoid(root='./data', name='cora')
   # dataset = Coauthor('./data/academic_cs', name='CS')
    feat_data = dataset[0].x.numpy()
    labels = np.expand_dims(dataset[0].y.numpy(), axis=1)
    edge = dataset[0].edge_index.numpy()


Processing...
Done!


In [5]:
feat_data.shape

(2708, 1433)

In [6]:
labels.shape

(2708, 1)

In [7]:
edge.shape

(2, 10556)

In [8]:
adj = sp.coo_matrix((np.ones(edge.shape[1]), (edge[0,:], edge[1,:])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

In [9]:
adj.nonzero()[0].shape


(10556,)

In [15]:
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj = torch.FloatTensor(np.array(adj.todense()))

TypeError: unsupported operand type(s) for +: 'Tensor' and 'dia_matrix'

In [11]:
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

In [12]:
adj_

NameError: name 'adj_' is not defined

In [13]:
adj

<2708x2708 sparse matrix of type '<class 'numpy.float32'>'
	with 10556 stored elements in Compressed Sparse Row format>

In [39]:
adj_[0][0]

TypeError: 'coo_matrix' object is not subscriptable

In [14]:
acc_list = []
for seed in range(10):
    adj, features, labels, idx_train, idx_val, idx_test,_ = load_data(seed=seed)
    model = SpGAT(nfeat=features.shape[1], 
                nhid=128, 
                nclass=int(labels.max()) + 1, 
                dropout=0, 
                nheads=8, 
                alpha=0.2)
    optimizer = optim.Adam(model.parameters(), 
                       lr=0.005, 
                       weight_decay=5e-4)
    features, adj, labels = Variable(features), Variable(adj), Variable(labels)
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()   
    
    
    
    t_total = time.time()
    loss_values = []
    bad_counter = 0
    best = 200 + 1
    best_epoch = 0
    for epoch in range(200):
        loss_values.append(train(epoch))
    
        torch.save(model.state_dict(), '{}.pkl'.format(epoch))
        if loss_values[-1] < best:
            best = loss_values[-1]
            best_epoch = epoch
            bad_counter = 0
        else:
            bad_counter += 1
    
        if bad_counter == 100:
            break
    
        files = glob.glob('*.pkl')
        for file in files:
            epoch_nb = int(file.split('.')[0])
            if epoch_nb < best_epoch:
                os.remove(file)
    
    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb > best_epoch:
            os.remove(file)
    
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    # Restore best model
    print('Loading {}th epoch'.format(best_epoch))
    model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))
    
    # Testing
    acc_list.append(compute_test())    
    

Loading cora dataset...


NameError: name 'train' is not defined

tensor([2, 5, 4,  ..., 1, 0, 2])

In [77]:
sum([0.90234375, 0.890625  , 0.85742188, 0.88476562, 0.8828125 ,
       0.89257812, 0.8828125 , 0.88476562, 0.8828125 , 0.875])

8.83593749

In [82]:
(acc_list)

[0.8872458410351202,
 0.88909426987061,
 0.8521256931608133,
 0.8964879852125693,
 0.8743068391866913,
 0.878003696857671,
 0.8706099815157117,
 0.8817005545286506,
 0.8632162661737522,
 0.8706099815157117]

In [62]:
adj, features, labels, idx_train, idx_val, idx_test = load_data(seed=1)

Loading cora dataset...


In [63]:
model = GAT(nfeat=features.shape[1], 
                nhid=128, 
                nclass=int(labels.max()) + 1, 
                dropout=0.6, 
                nheads=8, 
                alpha=0.2)
optimizer = optim.Adam(model.parameters(), 
                       lr=0.001, 
                       weight_decay=5e-4)

In [64]:
model = SpGAT(nfeat=features.shape[1], 
                nhid=128, 
                nclass=int(labels.max()) + 1, 
                dropout=0, 
                nheads=8, 
                alpha=0.2)
optimizer = optim.Adam(model.parameters(), 
                       lr=0.005, 
                       weight_decay=5e-4)

In [65]:
features, adj, labels = Variable(features), Variable(adj), Variable(labels)

In [1]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
#    print('Epoch: {:04d}'.format(epoch+1),
#          'loss_train: {:.4f}'.format(loss_train.data.item()),
#          'acc_train: {:.4f}'.format(acc_train.data.item()),
#          'loss_val: {:.4f}'.format(loss_val.data.item()),
#          'acc_val: {:.4f}'.format(acc_val.data.item()),
#          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()

In [8]:
def compute_test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    return acc_test.item()

In [68]:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [69]:
t_total = time.time()
loss_values = []
bad_counter = 0
best = 10000 + 1
best_epoch = 0
for epoch in range(200):
    loss_values.append(train(epoch))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == 100:
        break

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)

files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

# Testing
#compute_test()

Epoch: 0001 loss_train: 1.9457 acc_train: 0.1471 loss_val: 1.9436 acc_val: 0.1771 time: 0.0508s
Epoch: 0002 loss_train: 1.8819 acc_train: 0.4228 loss_val: 1.8809 acc_val: 0.4336 time: 0.0515s
Epoch: 0003 loss_train: 1.8215 acc_train: 0.4129 loss_val: 1.8215 acc_val: 0.4022 time: 0.0447s
Epoch: 0004 loss_train: 1.7615 acc_train: 0.4135 loss_val: 1.7627 acc_val: 0.4077 time: 0.0514s
Epoch: 0005 loss_train: 1.7010 acc_train: 0.4197 loss_val: 1.7033 acc_val: 0.4151 time: 0.0516s
Epoch: 0006 loss_train: 1.6396 acc_train: 0.4326 loss_val: 1.6436 acc_val: 0.4317 time: 0.0521s
Epoch: 0007 loss_train: 1.5776 acc_train: 0.4585 loss_val: 1.5838 acc_val: 0.4557 time: 0.0515s
Epoch: 0008 loss_train: 1.5148 acc_train: 0.4966 loss_val: 1.5238 acc_val: 0.4963 time: 0.0529s
Epoch: 0009 loss_train: 1.4509 acc_train: 0.5305 loss_val: 1.4637 acc_val: 0.5258 time: 0.0481s
Epoch: 0010 loss_train: 1.3856 acc_train: 0.5674 loss_val: 1.4030 acc_val: 0.5590 time: 0.0494s
Epoch: 0011 loss_train: 1.3187 acc_train

<All keys matched successfully>

In [70]:
compute_test()

Test set results: loss= 0.4073 accuracy= 0.8928


In [61]:
torch.cuda.empty_cache()

In [290]:
import numpy as np
import scipy.sparse as sp
import torch
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot
def cal_ego_weighted_m(m,beta):
    ego_weighted_list = []
    for row in m:
        new_row = (np.sum(adj[np.nonzero(row)[0]],axis=0)-1)*row
        ego_weighted_list.append(new_row)
    ego_weighted_arr = np.array(ego_weighted_list).astype(np.int).astype(np.float32)
    ego_weighted_arr = np.power(ego_weighted_arr,beta)
    ego_weighted_arr = sp.coo_matrix(ego_weighted_arr)
    return ego_weighted_arr
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [434]:
    
    idx_features_labels = np.genfromtxt("{}{}.content".format("/home/zhihao/Document/gnn_fd/graphSage/data/cora/raw/", "cora"),
                                        dtype=np.dtype(str))
    #features_1 = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    #labels_1 = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}

    edges_unordered = np.genfromtxt("{}{}.cites".format("/home/zhihao/Document/gnn_fd/graphSage/data/cora/raw/", "cora"),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = adj.toarray()
    adj = adj + np.eye(adj.shape[0],dtype=np.float32)
    adj = cal_ego_weighted_m(adj,0.6)
    
    adj = normalize(adj)
    adj = torch.FloatTensor(np.array(adj.todense()))

In [435]:
adj

tensor([[0.1823, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3135,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.3648, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.3648, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.3919]])